In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

In [ ]:
import numpy as np

# Load Foil Data

In [ ]:
from ICARUS.Input_Output.XFLR5.polars import read_polars_2d
from ICARUS.Database import DB
from ICARUS.Core.struct import Struct
from ICARUS.Database import EXTERNAL_DB

In [ ]:
read_polars_2d(EXTERNAL_DB)
airfoils: Struct = DB.foils_db.set_available_airfoils()

In [ ]:
airfoils_names = list(airfoils.keys())
print(airfoils_names)
DB.foils_db.get_airfoil_solvers(airfoil_name=airfoils_names[4])

# Get Plane

In [ ]:
# from examples.Planes.hermes import hermes
# airplane = hermes(airfoils ,"Hermes")

from ICARUS.Input_Output.XFLR5.parser import parse_xfl_project
filename: str = "Data/3d_Party/plane_1.xml"
airplane = parse_xfl_project(filename)
# airplane.name = "Embraer_E190_Cruise"
airplane.visualize()


# airplane = DB.vehicles_db.planes['hermes_3']
# airplane.save()

In [ ]:
# airplane.surfaces = [airplane.surfaces[i] for i in [2]]
# airplane.visualize()
airplane.airfoils

# Import Environment

In [ ]:
from ICARUS.Environment.definition import EARTH_ISA

# Get Solver

In [ ]:
from ICARUS.Solvers.Airplane.gnvp3 import get_gnvp3
gnvp3 = get_gnvp3()

# from ICARUS.Solvers.Airplane.gnvp7 import get_gnvp7
# gnvp7 = get_gnvp7()

# print(gnvp3)
print(gnvp3)

## AoA Run

In [ ]:
analysis: str = gnvp3.available_analyses_names()[2] 
print(f"Selecting Analysis: {analysis}")
gnvp3.set_analyses(analysis)
options: Struct = gnvp3.get_analysis_options(verbose=True)
solver_parameters: Struct = gnvp3.get_solver_parameters(verbose=True)

In [ ]:
AoAmin = -6
AoAmax = 8
NoAoA = (AoAmax - AoAmin) + 1
angles = np.linspace(AoAmin, AoAmax, NoAoA, dtype= float)
u_freestream = 20
# airplane.define_dynamic_pressure(u_freestream, EARTH.air_density)

options.plane.value         = airplane
options.environment.value   = EARTH_ISA
options.solver2D.value      = 'XFLR'
options.maxiter.value       = 100
options.timestep.value      = 0.001
options.u_freestream.value  = u_freestream
options.angles.value        = angles

solver_parameters.Use_Grid.value = 1

gnvp3.print_analysis_options()
gnvp3.run()
polars = gnvp3.get_results()
airplane.save()

# Dynamics

### Define and Trim Plane

In [ ]:
from ICARUS.Flight_Dynamics.state import State
if isinstance(polars,int):
    raise Exception("No Polars")
unstick = State("Unstick",airplane,polars,EARTH_ISA)

### Pertrubations

In [ ]:
unstick.add_all_pertrubations("Central")
unstick.get_pertrub()

In [ ]:
analysis = gnvp3.available_analyses_names(verbose=True)[4] # ANGLES PARALLEL
print(f"Selecting Analysis: {analysis}")
gnvp3.set_analyses(analysis)
options = gnvp3.get_analysis_options(verbose=True)

In [ ]:
options.plane.value         = airplane
options.state.value         = unstick
options.environment.value   = EARTH_ISA
options.solver2D.value      = 'XFLR'
options.maxiter.value       = 200
options.timestep.value      = 5e-3
options.u_freestream.value  = unstick.trim['U']
options.angle.value        = unstick.trim['AoA']

gnvp3.print_analysis_options()
gnvp3.run()
gnvp3.get_results()
unstick.save()

# Sensitivity Analysis

In [ ]:
# if sensAnalysisGNVP3 :
#     for var in ['u','w','q','theta','v','p','r','phi']:
#         space = np.logspace(np.log10(0.00001), np.log10(1), 10, base=10)
#         space =[*-space,*space]
#         maxiter = 2
#         timestep = 5e-2
#         dyn.sensitivityAnalysis(var, space)
#         GNVP3BatchArgs = [dyn, var, foildb, "Xfoil",
#                         maxiter, timestep,
#                         dyn.trim['U'], dyn.trim['AoA']]
#         dyn.runAnalysis(runGNVPsensitivityParallel, GNVP3BatchArgs)
#         perDIR = os.path.join(dyn.CASEDIR, f"Sensitivity_{var}")
#         dyn.sensResults[var] = pertrResults(perDIR, HOMEDIR)
#     dyn.save()